In [31]:
import pandas as pd
import numpy as np

In [32]:
Data = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/communities/communities.data',sep=',',header=None)

In [33]:
Data.shape

(1994, 128)

In [34]:
Data.head()

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,8,?,?,Lakewoodcity,1,0.19,0.33,0.02,0.90,0.12,...,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14,0.20
1,53,?,?,Tukwilacity,1,0.00,0.16,0.12,0.74,0.45,...,0.02,0.12,0.45,?,?,?,?,0.00,?,0.67
2,24,?,?,Aberdeentown,1,0.00,0.42,0.49,0.56,0.17,...,0.01,0.21,0.02,?,?,?,?,0.00,?,0.43
3,34,5,81440,Willingborotownship,1,0.04,0.77,1.00,0.08,0.12,...,0.02,0.39,0.28,?,?,?,?,0.00,?,0.12
4,42,95,6096,Bethlehemtownship,1,0.01,0.55,0.02,0.95,0.09,...,0.04,0.09,0.02,?,?,?,?,0.00,?,0.03


# Extract Feature and target/goal 

In [35]:
X = Data.loc[:,5:127]
print(X.shape)

(1994, 123)


# convert "?" label data to Nan

In [36]:
X = X.replace('?',np.nan)

In [37]:
X.dtypes.tail()

123     object
124     object
125    float64
126     object
127    float64
dtype: object

# need to change data types object to float in order to mean

In [38]:
X = X.convert_objects(convert_numeric=True)

g:\python\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  """Entry point for launching an IPython kernel.


In [39]:
X.dtypes.tail()

123    float64
124    float64
125    float64
126    float64
127    float64
dtype: object

# Get sample mean 

In [40]:
sample_mean = X.mean()
print(sample_mean.dtypes)
print(sample_mean.shape)

float64
(123,)


In [41]:
sample_mean.head()

5    0.057593
6    0.463395
7    0.179629
8    0.753716
9    0.153681
dtype: float64

In [42]:
ind = X.columns
ind

Int64Index([  5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
            ...
            118, 119, 120, 121, 122, 123, 124, 125, 126, 127],
           dtype='int64', length=123)

# replace Nan with sample mean

In [43]:
for i in range (0,ind.shape[0]):
    X.loc[:,ind[i]].replace(np.nan,sample_mean[ind[i]],inplace=True)

# convert in to numpy array 

In [44]:
X = pd.DataFrame.as_matrix(X)
print(X.shape)
print(type(X))

(1994, 123)
<class 'numpy.ndarray'>


In [45]:
X.shape[0]

1994

In [46]:
# no of files for validation at per iteration
# X.shape[0] is the number of total examples
test_data_itr = int(np.ceil(X.shape[0]/5))
print(test_data_itr)

399


In [47]:
def createData (filetype,filenumber,content):
    import numpy as np
    np.savetxt("F:/Winter 2018/Applied ML/winter 2018/Assignments/assignment 1/codes/CandC-%s%s.csv" % (filetype,filenumber) , content, delimiter=",")


In [48]:
for i in range (0,5):
    strt_ind = i*test_data_itr
    if i == 4 :
        X_test = X[strt_ind:,:]
        X_train = np.delete(X,np.s_[strt_ind :],0)
    else :
        X_test = X[strt_ind :strt_ind + test_data_itr,:]
        X_train = np.delete(X,np.s_[strt_ind :strt_ind + test_data_itr],0)
    
    createData ('train',i,X_train)
    createData ('test',i,X_test)
    
    print(X_test.shape)
    print(X_train.shape)
    

(399, 123)
(1595, 123)
(399, 123)
(1595, 123)
(399, 123)
(1595, 123)
(399, 123)
(1595, 123)
(398, 123)
(1596, 123)


In [50]:
###############################################################################################################################




###############################################################################################################################

In [56]:
def getfile (filename):
    blah = pd.read_csv(filename,sep=',',header=None)
    blah = pd.DataFrame.as_matrix(blah)
    return blah

In [51]:
def computeW (Xm_train,Y_train):
    a = np.dot(Xm_train.T,Xm_train) 
    a = np.linalg.pinv(a)
    W = np.dot(a, np.dot(Xm_train.T,Y_train))
    return W

In [52]:
def computeMSE (Hypothesis,Y):
    MSE = np.mean((Hypothesis-Y)**2) 
    return MSE

In [53]:
trainMSE = []
testMSE = []
for fold in range (0,5):
    trainData = getfile("F:/Winter 2018/Applied ML/winter 2018/Assignments/assignment 1/codes/CandC-train%s.csv" %fold)
    testData = getfile("F:/Winter 2018/Applied ML/winter 2018/Assignments/assignment 1/codes/CandC-test%s.csv" %fold)


    # separate the labels from features
    trainFeatures = trainData[:,0:-1]
    trainLabel = trainData[:,-1]
    trainLabel = np.reshape(trainLabel,(-1,1))

    testFeatures = testData[:,0:-1]
    testLabel = testData[:,-1]
    testLabel = np.reshape(testLabel,(-1,1))

    # compute W
    W = computeW (trainFeatures,trainLabel)

    # find approxiamte value
    trainHypothesis = np.dot(trainFeatures,W)
    testHypothesis = np.dot(testFeatures,W)

    # find MSE (mean square error)
    trainMSE.append ( computeMSE (trainHypothesis,trainLabel) )
    testMSE.append ( computeMSE (testHypothesis,testLabel) )
    
    print(fold)

0
1
2
3
4


In [54]:
trainMSE

[0.01569994063447588,
 0.014794852348657591,
 0.01659717705739191,
 0.01673020389318852,
 0.01615744737001229]

In [55]:
testMSE

[0.020252594515740605,
 0.02519236700980506,
 0.016812226084418774,
 0.016320964734337112,
 0.867820306347294]

In [57]:
###############################################################################################################################


###############################################################################################################################

In [66]:
def computeW (Xm_train,Y_train,no_features,lmbda):
    import numpy as np
    a = np.dot(Xm_train.T,Xm_train) + lmbda*np.eye(no_features)
    a = np.linalg.pinv(a)
    W = np.dot(a, np.dot(Xm_train.T,Y_train))
    return W

In [59]:
def computeMSE (Hypothesis,Y,lmbda,W):
    MSE = np.mean((Hypothesis-Y)**2) + ( lmbda * (np.dot(W.T,W)) )
    return MSE

In [ ]:
MSE_traindata = []
MSE_validdata = []

for i in range (0,11):
    lmbda = i/10
    
    W = computeW (Xm_train,Y_train,feature_no,lmbda)
    
    Hypothesis_train = np.dot(Xm_train,W)
    Hypothesis_valid = np.dot(Xm_valid,W)
    
    MSE_traindata = np.append ( computeMSE (Hypothesis_train,Y_train,lmbda,W), MSE_traindata )
    MSE_validdata = np.append ( computeMSE (Hypothesis_valid,Y_valid,lmbda,W), MSE_validdata )

In [73]:
trainMSE = []
testMSE = []
lmbda = 0
for fold in range (0,5):
    trainData = getfile("F:/Winter 2018/Applied ML/winter 2018/Assignments/assignment 1/codes/CandC-train%s.csv" %fold)
    testData = getfile("F:/Winter 2018/Applied ML/winter 2018/Assignments/assignment 1/codes/CandC-test%s.csv" %fold)


    # separate the labels from features
    trainFeatures = trainData[:,0:-1]
    trainLabel = trainData[:,-1]
    trainLabel = np.reshape(trainLabel,(-1,1))

    testFeatures = testData[:,0:-1]
    testLabel = testData[:,-1]
    testLabel = np.reshape(testLabel,(-1,1))

    # compute W
    no_features = trainFeatures.shape[1]
    W = computeW (trainFeatures,trainLabel,no_features,lmbda)

    # find approxiamte value
    trainHypothesis = np.dot(trainFeatures,W)
    testHypothesis = np.dot(testFeatures,W)

    # find MSE (mean square error)
    trainMSE.append ( computeMSE (trainHypothesis,trainLabel,lmbda,W) )
    testMSE.append ( computeMSE (testHypothesis,testLabel,lmbda,W) )
    
    print(fold)

0
1
2
3
4


In [74]:
 trainMSE

[array([[0.01569994]]),
 array([[0.01479485]]),
 array([[0.01659718]]),
 array([[0.0167302]]),
 array([[0.01615745]])]

In [75]:
 testMSE

[array([[0.02025259]]),
 array([[0.02519237]]),
 array([[0.01681223]]),
 array([[0.01632096]]),
 array([[0.86782031]])]

In [68]:
no_features

123